In [ ]:
import rebound
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def setup_sim():
    sim = rebound.Simulation()
    sim.units = ('AU', 'days', 'Msun')
    # We can add Jupiter and four of its moons by name, since REBOUND is linked to the HORIZONS database.
    labels = ["Jupiter", "Io", "Europa"]
    #sim.add(labels)

    sim.add(m=0.000954791915211)
    sim.add(m=4.4908495628043e-8,a=0.00281895041549, e=0.00410976900750*2.2, inc=0.0383464557686*0., Omega=-0.377395916998, omega=-0.581381978207, f=-1.823050122780)
    sim.add(m=2.4132920575575e-8,a=0.00448620889471, e=0.00921827235167*0.05, inc=0.0468742357719*0., Omega=-0.400332782240, omega= 2.581184196550, f=-0.910276392336)
    os = sim.calculate_orbits()
    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = 0.02 * os[0].P  # 2% of Io's period
    return sim
sim = setup_sim()
os = sim.calculate_orbits()

In [ ]:
Nout = 50000          # number of points to display
tmax = 40*365.25         # let the simulation run for 40 years
Nmoons = 2
print "Io's period is {p}".format(p=os[0].P)

In [ ]:
def bisect(sim,t1,t2):
    tm = (t1+t2)/2.
    if t2-t1<(5.0e-7):
        return tm
    sim.integrate(tm)
    os = sim.calculate_orbits()
    if (os[0].theta - os[1].theta)>0.:
        t2 = tm
        return bisect(sim,t1,t2)
    else:
        t1 = tm
        return bisect(sim,t1,t2)

In [ ]:
def setup_reoriented_sim(xlist, vxlist):
    if(xlist[2][0]>0):
        rotation_angle = np.arccos((-1. * xlist[2][1])/((xlist[2][0]**2.+xlist[2][1]**2.)**(0.5)))
    else:
        rotation_angle = -np.arccos((-1. * xlist[2][1])/((xlist[2][0]**2.+xlist[2][1]**2.)**(0.5)))
    c, s = np.cos(rotation_angle), np.sin(rotation_angle)
    R = [[c,-s],[s,c]]
    
    sim = rebound.Simulation()
    sim.units = ('AU', 'days', 'Msun')
    masses = [0.000954791915211,4.4908495628043e-8,2.4132920575575e-8]
    for i in range(3):
        xy, vxy = np.ones(2), np.ones(2)
        xy[0], xy[1] = xlist[i][0], xlist[i][1]
        vxy[0], vxy[1] = vxlist[i][0], vxlist[i][1]
        r_xy = np.dot(xy,R)
        r_vxy = np.dot(vxy,R)
        sim.add(m=masses[i], x=r_xy[0], y=r_xy[1], vx=r_vxy[0], vy=r_vxy[1])
    sim.move_to_com()
    return sim

In [ ]:
List_of_all_sims = []
List_of_all_ecc = []
List_of_all_sim_times = []

import matplotlib.gridspec as gridspec
import gc
sim = setup_sim()
sim_times = []
delta_angle_old = 0.1
prev_time = 0.
iteration = 0
x = np.zeros((Nmoons,Nout))
ecc = np.zeros((Nmoons,Nout))
longitude = np.zeros((Nmoons,Nout))
varpi = np.zeros((Nmoons,Nout))
ps = sim.particles

while(prev_time < tmax):
    sim.step()
    os = sim.calculate_orbits()
    delta_angle_new = os[0].theta - os[1].theta
    if(delta_angle_new>0 and delta_angle_old<0):
        time_conjunction = bisect(sim, prev_time, sim.t)
        os = sim.calculate_orbits()
        if(np.isclose([os[0].theta],[os[1].theta],atol=1e-5,rtol=0.0)):
            sim_times.append(time_conjunction)
            #Rotating
            cartesian_xlist = []
            cartesian_vxlist = []
            for i in range(3):
                cartesian_xlist.append(sim.particles[i].xyz)
                cartesian_vxlist.append(sim.particles[i].vxyz)
            sim_temp = setup_reoriented_sim(cartesian_xlist, cartesian_vxlist)
            #commented code is for printing frames while running (much ram required). OrbitPlot modified to take fig and axes as argument.
            #fig = plt.figure(figsize=[6,9])
            #fig.suptitle("2:1 Mean Motion Resonance, at Conjunctions in Rotating Frame", fontsize=12)
            #gs = gridspec.GridSpec(3,1)
            #ax1 = plt.subplot(gs[:2, :])
            #fakeforlegend = plt.Line2D((0,1),(0,0), color='k', linestyle='dotted', label='Periastron')
            #ax1.legend([fakeforlegend], ['Periastron'], loc='upper right',prop={'size': 9})
            #dummy = rebound.OrbitPlot(sim_temp, trails=True, periastron=True, passedFig=fig, passedAx=ax1) #fig.axes[0].axis('off') # turn off axes
            
            #Recording
            for j in range(Nmoons):
                x[j][iteration] = ps[j+1].x
                ecc[j][iteration] = os[j].e
                longitude[j][iteration] = os[j].l
                varpi[j][iteration] = os[j].Omega + os[j].omega
            
            #frame2=fig.add_axes((.1,.1,.8,.2))
            #ax2 = plt.subplot(gs[2, :])
            #plt.xlim([0, tmax])
            #plt.ylim([0,0.01])
            #ax2.set_xlabel("Time")
            #ax2.set_ylabel("Eccentricity")
            #plt.plot(sim_times[:iteration], ecc[0][:iteration], label="Inner planet eccentricity")
            #plt.plot(sim_times[:iteration], ecc[1][:iteration], label="Outer planet eccentricity")
            #plt.legend(loc='upper right',prop={'size': 9})
            #fig.savefig('tmp/pngs_conj_e/{0:0=5d}.png'.format(iteration))
            #plt.close('all')
            
            #record all data for frames - more memory efficient (somehow).
            List_of_all_sims.append(sim_temp)
            List_of_all_ecc.append(np.copy(ecc))
            List_of_all_sim_times.append(np.copy(sim_times))
            #Post step
            iteration += 1
            if(iteration%100==0):
                print "Iteration:{a}".format(a=iteration)
            sim.step()
            os = sim.calculate_orbits()
    delta_angle_old = os[0].theta - os[1].theta
    prev_time = sim.t

In [ ]:
print len(List_of_all_sims)
#0 to 4104, p from 0 to 10
p = 10
for i in range(((p)*400),((p+1)*400)):
    sim_temp = List_of_all_sims[i]
    sim_times = List_of_all_sim_times[i]
    ecc = List_of_all_ecc[i]
    
    fig = plt.figure(figsize=[6,9])
    fig.suptitle("2:1 Mean Motion Resonance, at Conjunctions in Rotating Frame", fontsize=12)
    gs = gridspec.GridSpec(3,1)
    ax1 = plt.subplot(gs[:2, :])
    fakeforlegend = plt.Line2D((0,1),(0,0), color='k', linestyle='dotted', label='Periastron')
    ax1.legend([fakeforlegend], ['Periastron'], loc='upper right',prop={'size': 9})
    dummy = rebound.OrbitPlot(sim_temp, trails=True, periastron=True, passedFig=fig, passedAx=ax1) #fig.axes[0].axis('off') # turn off axes
    ax2 = plt.subplot(gs[2, :])
    plt.xlim([0, tmax])
    plt.ylim([0,0.01])
    ax2.set_xlabel("Time")
    ax2.set_ylabel("Eccentricity")
    plt.plot(sim_times[:i], ecc[0][:i], label="Inner planet eccentricity")
    plt.plot(sim_times[:i], ecc[1][:i], label="Outer planet eccentricity")
    plt.legend(loc='upper right',prop={'size': 9})
    fig.savefig('tmp/pngs_conj_e/{0:0=5d}.png'.format(i))
    plt.close('all')
    if(i%20==0):
        print i
print "Done, safe to restart"

In [ ]:
import glob
list_all_files = glob.glob('tmp/pngs_conj_e/*.png')
print len(list_all_files)

In [ ]:
'''ffmpeg -f image2 -r 60 -i %05d.png -crf 16 -vcodec libx264 -y movie2_60fps.mp4
'''